# Completeness of Scholars@TAMU Data After Cleaning  
**Filename:** completeness_after_clean.ipynb  
**Path:** TAMIDS/Code/Scholars@TAMU Data/completeness_after_clean.ipynb  
**Created Date:** 02 April 2022, 21:30 

This is pretty much the same file as before, but using the data produced by cleaning.ipynb

In [1]:
from IPython.display import Markdown, display, HTML
from typing import Dict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json

pd.options.display.float_format = '{:,.3f}'.format
pd.options.styler.format.precision = 2
pd.options.styler.format.thousands = ','
pd.options.display.max_columns = 50
pd.options.display.max_rows = 150
plt.style.use('seaborn-darkgrid')

# General Markdown Formatting Functions

def printmd(string, level=1):
    header_level = '#'*level + ' '
    display(Markdown(header_level + string))

## Scholars@TAMU

In [2]:
base_path = "../../Data/Scholars@TAMU"

!ls "$base_path"

courses           my_api_calls      people
grants_and_awards organizations     publications


In [3]:
def count_types(df: pd.DataFrame, percentage=False, plot=False, show_table=False, color_map:str=False) -> pd.DataFrame:
    new_df = pd.DataFrame()

    # the below can count the total amount of characters in a column
    # doof = df.copy()
    # doof = doof.applymap(lambda x: len(str(x)))
    # display(doof.sum())

    for column in df:
        new_df[column] = df[column].apply(lambda x: "nan" if pd.isna(x) else type(x).__name__)
    new_df = new_df.apply(pd.value_counts)

    if percentage:
        new_df = new_df / new_df.sum() * 100

    if plot:
        new_df.T.plot.bar()
        plt.yscale("log")
        plt.rcParams["figure.figsize"]
        plt.show()

    if show_table:
        if color_map:
            cm = sns.light_palette(color_map, as_cmap=True)
            color_df = new_df.style.background_gradient(cmap=cm, axis=None)
            color_df = color_df.applymap(lambda x: 'color: transparent' if pd.isnull(x) else '')
            color_df = color_df.applymap(lambda x: 'background-color: transparent' if pd.isnull(x) else '')
            display(color_df)
        else:
            display(new_df)


    return new_df


In [4]:
def slice_unnamed_columns(df: pd.DataFrame):
    return df.loc[:,~df.columns.str.contains('Unnamed:')]

def calculate_completeness(df: pd.DataFrame, show_table=False, color_map:str=False) -> pd.DataFrame:
    sliced_df = slice_unnamed_columns(df)
    missing_values = sliced_df.isna().sum()
    total_rows = sliced_df.shape[0]
    availible_values = sliced_df.count()
    data_completeness = (1 - missing_values / total_rows ) *  100

    new_df = pd.DataFrame({
        'data_completeness': data_completeness,
        'total_rows': total_rows,
        'availible_values': availible_values,
        'missing_values': missing_values
    })

    if show_table:
        if color_map:
            cm = sns.color_palette(color_map, as_cmap=True)
            color_df = new_df.style.text_gradient(cmap=cm, axis=0, gmap=new_df['data_completeness'], vmin=0, vmax=100)
            display(color_df)
        else:
            display(new_df)

    return new_df

In [5]:
def col_types_to_json(df_dict:Dict[str, pd.DataFrame], folder_name: str):
    type_dict = {}
    for file_name, df in df_dict.items():
        sliced_df = slice_unnamed_columns(df)
        counted_df = count_types(sliced_df)
        type_dict[file_name] = {column: counted_df[column].idxmax() for column in counted_df}

    with open('dicts/column_types/' + folder_name + '_col_types.json', 'w') as outfile:
        json.dump(type_dict, outfile, indent=4)

In [6]:
with open('dicts/data_filenames.json', 'r') as outfile:
    data_filenames = json.load(outfile)

## ./people

In [7]:
people_path = base_path + "/people"

people_data = {picklename: pd.read_pickle(people_path + "/" + picklename + ".pickle") for picklename in data_filenames['people']}

In [8]:
for name, df in people_data.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### people_affiliation

,uid,uin,people_uri,people_api,position_dept_id
float,nan,nan,nan,nan,"5,912.00"
int,nan,"5,944.00",nan,nan,nan
nan,nan,nan,nan,nan,32.00
str,"5,944.00",nan,"5,944.00","5,944.00",nan


### people_education

,uid,uin,people_uri,people_api,external_org_id,granting_school_name,degreeuri_text,year,major
float,nan,"9,118.00",nan,nan,"9,120.00",nan,nan,nan,nan
int,nan,nan,nan,nan,nan,nan,nan,"8,612.00",nan
nan,3.00,3.00,3.00,3.00,1.00,nan,459.00,509.00,"1,398.00"
str,"9,118.00",nan,"9,118.00","9,118.00",nan,"9,121.00","8,662.00",nan,"7,723.00"


### people_overview

,uid,uin,people_uri,people_api,status,dept_id,lastname,middle,firstname,preferred_title,email,overview
float,nan,nan,nan,nan,nan,"4,859.00",nan,nan,nan,nan,nan,nan
int,nan,"4,859.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,"1,531.00",nan,5.00,129.00,"3,063.00"
str,"4,859.00",nan,"4,859.00","4,859.00","4,859.00",nan,"4,859.00","3,328.00","4,859.00","4,854.00","4,730.00","1,796.00"


### people_subject_areas

,uid,uin,people_uri,people_api,research_areas
int,nan,"7,328.00",nan,nan,nan
nan,nan,nan,nan,nan,"4,226.00"
str,"7,328.00",nan,"7,328.00","7,328.00","3,102.00"


In [9]:
people_stats = {}

for name, df in people_data.items():
    printmd(name, 3)
    people_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### people_affiliation

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"5,944","5,944",0
uin,100.00,"5,944","5,944",0
people_uri,100.00,"5,944","5,944",0
people_api,100.00,"5,944","5,944",0
position_dept_id,99.46,"5,944","5,912",32


### people_education

,data_completeness,total_rows,availible_values,missing_values
uid,99.97,"9,121","9,118",3
uin,99.97,"9,121","9,118",3
people_uri,99.97,"9,121","9,118",3
people_api,99.97,"9,121","9,118",3
external_org_id,99.99,"9,121","9,120",1
granting_school_name,100.00,"9,121","9,121",0
degreeuri_text,94.97,"9,121","8,662",459
year,94.42,"9,121","8,612",509
major,84.67,"9,121","7,723","1,398"


### people_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"4,859","4,859",0
uin,100.00,"4,859","4,859",0
people_uri,100.00,"4,859","4,859",0
people_api,100.00,"4,859","4,859",0
status,100.00,"4,859","4,859",0
dept_id,100.00,"4,859","4,859",0
lastname,100.00,"4,859","4,859",0
middle,68.49,"4,859","3,328","1,531"
firstname,100.00,"4,859","4,859",0
preferred_title,99.90,"4,859","4,854",5


### people_subject_areas

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"7,328","7,328",0
uin,100.00,"7,328","7,328",0
people_uri,100.00,"7,328","7,328",0
people_api,100.00,"7,328","7,328",0
research_areas,42.33,"7,328","3,102","4,226"


## ./courses

In [10]:
courses_path = base_path + "/courses"

courses_data = {picklename: pd.read_pickle(courses_path + "/" + picklename + ".pickle") for picklename in data_filenames['courses']}

In [11]:
for name, df in courses_data.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### course_overview

,uid,uin,people_uri,people_api,course,course_name,acad_dept,year
int,nan,"70,291.00",nan,nan,nan,nan,nan,"70,258.00"
nan,nan,nan,nan,nan,nan,nan,nan,33.00
str,"70,291.00",nan,"70,291.00","70,291.00","70,291.00","70,291.00","70,291.00",nan


In [12]:
courses_stats = {}

for name, df in courses_data.items():
    printmd(name, 3)
    courses_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### course_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"70,291","70,291",0
uin,100.00,"70,291","70,291",0
people_uri,100.00,"70,291","70,291",0
people_api,100.00,"70,291","70,291",0
course,100.00,"70,291","70,291",0
course_name,100.00,"70,291","70,291",0
acad_dept,100.00,"70,291","70,291",0
year,99.95,"70,291","70,258",33


## ./grants_and_awards

In [13]:
grants_path = base_path + "/grants_and_awards"

grants_data = {picklename: pd.read_pickle(grants_path + "/" + picklename + ".pickle") for picklename in data_filenames['grants_and_awards']}

In [14]:
for name, df in grants_data.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### awards_overview

,uid,uin,people_uri,people_api,award_id,year,award_name,authoritative_name_id
float,nan,nan,nan,nan,"4,198.00","4,198.00",nan,"4,192.00"
int,nan,"10,052.00",nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,"5,854.00","5,854.00","5,860.00","5,860.00"
str,"10,052.00",nan,"10,052.00","10,052.00",nan,nan,"4,192.00",nan


### grants_abstract

,grant_uri,grant_api,abstract
nan,nan,nan,"2,970"
str,"5,802.00","5,802.00","2,832"


### grants_overview

,uid,uin,people_uri,people_api,grant_uid,grant_uri,grant_api,start_year,end_year,role,grant_awarded_by_external_org_id,title
float,nan,nan,nan,nan,nan,nan,nan,"5,752.00","5,729.00",nan,"5,800.00",nan
int,nan,"5,803.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,51.00,74.00,nan,3.00,nan
str,"5,803.00",nan,"5,803.00","5,803.00","5,803.00","5,803.00","5,803.00",nan,nan,"5,803.00",nan,"5,803.00"


In [15]:
grants_stats = {}

for name, df in grants_data.items():
    printmd(name, 3)
    grants_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### awards_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"10,052","10,052",0
uin,100.00,"10,052","10,052",0
people_uri,100.00,"10,052","10,052",0
people_api,100.00,"10,052","10,052",0
award_id,41.76,"10,052","4,198","5,854"
year,41.76,"10,052","4,198","5,854"
award_name,41.70,"10,052","4,192","5,860"
authoritative_name_id,41.70,"10,052","4,192","5,860"


### grants_abstract

,data_completeness,total_rows,availible_values,missing_values
grant_uri,100.00,"5,802","5,802",0
grant_api,100.00,"5,802","5,802",0
abstract,48.81,"5,802","2,832","2,970"


### grants_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"5,803","5,803",0
uin,100.00,"5,803","5,803",0
people_uri,100.00,"5,803","5,803",0
people_api,100.00,"5,803","5,803",0
grant_uid,100.00,"5,803","5,803",0
grant_uri,100.00,"5,803","5,803",0
grant_api,100.00,"5,803","5,803",0
start_year,99.12,"5,803","5,752",51
end_year,98.72,"5,803","5,729",74
role,100.00,"5,803","5,803",0


## ./organizations

In [16]:
orgs_path = base_path + "/organizations"

orgs_data = {picklename: pd.read_pickle(orgs_path + "/" + picklename + ".pickle") for picklename in data_filenames['organizations']}

In [17]:
for name, df in orgs_data.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### organizations_external

,external_org_id,external_org_uri,external_org_api,authoritative_name,authoritative_name_city,authoritative_name_state,authoritative_name_country,grid_id,aa_id
int,"95,673.00",nan,nan,nan,nan,nan,nan,nan,993.00
nan,nan,nan,nan,1.00,473.00,"60,626.00",450.00,532.00,"94,680.00"
str,nan,"95,673.00","95,673.00","95,672.00","95,200.00","35,047.00","95,223.00","95,141.00",nan


### organizations_local

,[dept_id],[dept_name],[department_uri],[department_api]
int,586.00,nan,nan,nan
str,nan,586.00,586.00,586.00


In [18]:
orgs_stats = {}

for name, df in orgs_data.items():
    printmd(name, 3)
    orgs_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### organizations_external

,data_completeness,total_rows,availible_values,missing_values
external_org_id,100.00,"95,673","95,673",0
external_org_uri,100.00,"95,673","95,673",0
external_org_api,100.00,"95,673","95,673",0
authoritative_name,100.00,"95,673","95,672",1
authoritative_name_city,99.51,"95,673","95,200",473
authoritative_name_state,36.63,"95,673","35,047","60,626"
authoritative_name_country,99.53,"95,673","95,223",450
grid_id,99.44,"95,673","95,141",532
aa_id,1.04,"95,673",993,"94,680"


### organizations_local

,data_completeness,total_rows,availible_values,missing_values
[dept_id],100.00,586,586,0
[dept_name],100.00,586,586,0
[department_uri],100.00,586,586,0
[department_api],100.00,586,586,0


## ./publications

In [19]:
pubs_path = base_path + "/publications"

pubs_data = {picklename: pd.read_pickle(pubs_path + "/" + picklename + ".pickle") for picklename in data_filenames['publications']}

In [20]:
for name, df in pubs_data.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### publications_abstract

,publication_uri,publication_api,abstract
nan,nan,nan,"55,692"
str,"223,178.00","223,178.00","167,486"


### publications_author_institutions

,publication_uri,publication_api,organisation,city,country
nan,nan,nan,1,nan,nan
str,"950,084.00","950,084.00","950,083","950,084.00","950,084.00"


### publications_overview

,people_uid,uin,people_uri,people_api,dept_id,publication_uid,publication_uri,publication_api,doi,issn,eissn,journal_title,month,year,begin_page,end_page,volume,issue,publisher,publication_type,publication_title
int,nan,"223,171.00",nan,nan,"223,171.00",nan,nan,nan,nan,nan,nan,nan,"211,516.00","222,634.00",nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,"43,510.00","34,803.00","81,816.00","28,574.00","11,655.00",537.00,nan,nan,nan,nan,"41,307.00",nan,15.00
str,"223,171.00",nan,"223,171.00","223,171.00",nan,"223,171.00","223,171.00","223,171.00","179,661.00","188,368.00","141,355.00","194,597.00",nan,nan,"223,171.00","223,171.00","223,171.00","223,171.00","181,864.00","223,171.00","223,156.00"


### publications_subject_journal_wos

,publication_uri,publication_api,wos_id,wos_research_area_id,keyword
float,nan,nan,nan,"194,946.00",nan
nan,nan,nan,"12,475.00","12,475.00","12,475.00"
str,"207,421.00","207,421.00","194,946.00",nan,"194,946.00"


### publications_unsdg

,publication_uri,publication_api,category_sdg_id,name
int,nan,nan,"14,154.00",nan
str,"14,154.00","14,154.00",nan,"14,154.00"


In [21]:
pubs_stats = {}

for name, df in pubs_data.items():
    printmd(name, 3)
    pubs_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### publications_abstract

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"223,178","223,178",0
publication_api,100.00,"223,178","223,178",0
abstract,75.05,"223,178","167,486","55,692"


### publications_author_institutions

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"950,084","950,084",0
publication_api,100.00,"950,084","950,084",0
organisation,100.00,"950,084","950,083",1
city,100.00,"950,084","950,084",0
country,100.00,"950,084","950,084",0


### publications_overview

,data_completeness,total_rows,availible_values,missing_values
people_uid,100.00,"223,171","223,171",0
uin,100.00,"223,171","223,171",0
people_uri,100.00,"223,171","223,171",0
people_api,100.00,"223,171","223,171",0
dept_id,100.00,"223,171","223,171",0
publication_uid,100.00,"223,171","223,171",0
publication_uri,100.00,"223,171","223,171",0
publication_api,100.00,"223,171","223,171",0
doi,80.50,"223,171","179,661","43,510"
issn,84.41,"223,171","188,368","34,803"


### publications_subject_journal_wos

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"207,421","207,421",0
publication_api,100.00,"207,421","207,421",0
wos_id,93.99,"207,421","194,946","12,475"
wos_research_area_id,93.99,"207,421","194,946","12,475"
keyword,93.99,"207,421","194,946","12,475"


### publications_unsdg

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"14,154","14,154",0
publication_api,100.00,"14,154","14,154",0
category_sdg_id,100.00,"14,154","14,154",0
name,100.00,"14,154","14,154",0


## Combined

In [22]:
combined_dict = {
    'people': pd.concat(people_stats.values(), axis=0, keys=people_stats.keys()),
    'courses': pd.concat(courses_stats.values(), axis=0, keys=courses_stats.keys()),
    'grants_and_awards': pd.concat(grants_stats.values(), axis=0, keys=grants_stats.keys()),
    'organizations': pd.concat(orgs_stats.values(), axis=0, keys=orgs_stats.keys()),
    'publications': pd.concat(pubs_stats.values(), axis=0, keys=pubs_stats.keys())
}

combined_df = pd.concat(combined_dict.values(), axis=0, keys=combined_dict.keys())

combined_df

data_completeness  \
people            people_affiliation               uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   position_dept_id                             99.462   
                  people_education                 uid                                          99.967   
                                                   uin                                          99.967   
                                                   people_uri                                   99.967   
                                                   people_api                                   99.967   
                                                   external_org_id                              99.989   
                                                   granting_school_name                        100.000   
                                                   degreeuri_text                               94.968   
                                                   year                                         94.419   
                                                   major                                        84.673   
                  people_overview                  uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   status                                      100.000   
                                                   dept_id                                     100.000   
                                                   lastname                                    100.000   
                                                   middle                                       68.491   
                                                   firstname                                   100.000   
                                                   preferred_title                              99.897   
                                                   email                                        97.345   
                                                   overview                                     36.962   
                  people_subject_areas             uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   research_areas                               42.331   
courses           course_overview                  uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   course                                      100.000   
                                                   course_name                                 100.000   
                                                   acad_d